# TD1 Elasticsearch - Guide Explicatif

## Prérequis

Assurez-vous d'avoir installé la librairie :
```bash
pip install elasticsearch
```

Et que votre cluster Elasticsearch est démarré sur `http://localhost:9200`


In [ ]:
# Import de la librairie et connexion au cluster
from elasticsearch import Elasticsearch
from pprint import pprint

# Connexion au cluster Elasticsearch
es = Elasticsearch("http://localhost:9200")

print("✅ Connexion établie avec Elasticsearch")


## 0. Vérification de la santé du cluster

Vérifions que le cluster Elasticsearch fonctionne correctement.


In [ ]:
# Vérification de la santé du cluster
health = es.cluster.health()
pprint(health)


## 0. Fix des paramètres d'index en lecture seule

Si un index est en mode lecture seule, il faut d'abord désactiver ce paramètre avant de pouvoir le modifier.


In [ ]:
# Désactiver le mode lecture seule pour l'index tweet (si nécessaire)
# Cette commande permet de modifier un index qui serait bloqué en lecture seule
try:
    es.indices.put_settings(
        index="tweet",
        body={
            "index.blocks.read_only_allow_delete": None
        }
    )
    print("✅ Paramètres d'index mis à jour")
except Exception as e:
    print(f"ℹ️  Index peut-être inexistant ou déjà modifiable: {e}")


## 1. Création d'un index

Créons un index nommé "tweet" avec des paramètres spécifiques (nombre de shards et répliques).


In [ ]:
# Supprimer l'index s'il existe déjà (pour repartir à zéro)
if es.indices.exists(index="tweet"):
    es.indices.delete(index="tweet")
    print("🗑️  Index 'tweet' supprimé")

# Créer un nouvel index avec des paramètres spécifiques
es.indices.create(
    index="tweet",
    body={
        "settings": {
            "index": {
                "number_of_shards": 1,
                "number_of_replicas": 1
            }
        }
    }
)
print("✅ Index 'tweet' créé avec succès")


## 2. Définition du mapping

Le mapping définit la structure des documents qui seront indexés. Ici, nous définissons trois champs :
- `user_name` : de type `keyword` (recherche exacte)
- `content` : de type `text` (recherche full-text)
- `tweeted_at` : de type `date`


In [ ]:
# Supprimer et recréer l'index avec le mapping
# Note: Pour modifier le mapping d'un index existant, il faut le supprimer et le recréer
if es.indices.exists(index="tweet"):
    es.indices.delete(index="tweet")
    print("🗑️  Ancien index supprimé")

# Créer l'index avec le mapping
es.indices.create(
    index="tweet",
    body={
        "mappings": {
            "properties": {
                "user_name": {"type": "keyword"},
                "content": {"type": "text"},
                "tweeted_at": {"type": "date"}
            }
        }
    }
)
print("✅ Index 'tweet' créé avec le mapping défini")


## 3. Ajout d'un document JSON

Ajoutons un premier document dans l'index. Avec `POST`, Elasticsearch génère automatiquement un ID unique.


In [ ]:
# Ajouter un document (POST génère automatiquement un ID)
response = es.index(
    index="tweet",
    document={
        "user_name": "Trump",
        "content": "THE USA IS GREAT!",
        "tweeted_at": "2019-01-10"
    }
)
print("✅ Document ajouté avec succès")
pprint(response)


## 4. Insertion de multiples documents (Bulk)

La méthode `bulk` permet d'insérer plusieurs documents en une seule opération, ce qui est beaucoup plus efficace que d'insérer les documents un par un.


In [ ]:
# Préparer les actions bulk au format NDJSON (Newline Delimited JSON)
# Format: une ligne d'action, suivie d'une ligne de document
import json

actions = [
    {"index": {"_index": "tweet", "_id": "1"}},
    {"user_name": "Poutine", "content": "You are so 2019.. now it's Russia!", "tweeted_at": "2022-03-14"},
    {"index": {"_index": "tweet", "_id": "2"}},
    {"user_name": "Zelensky", "content": "Russia doesn't mean Ukraine...", "tweeted_at": "2022-03-15"},
    {"index": {"_index": "tweet", "_id": "3"}},
    {"user_name": "Trump", "content": "Sorry guys, I can't play with you, I'm retired now", "tweeted_at": "2022-03-16"}
]

# Convertir en format NDJSON (chaque ligne est un JSON séparé)
bulk_body = "\n".join([json.dumps(action) for action in actions]) + "\n"

# Exécuter l'opération bulk
response = es.bulk(body=bulk_body)
print(f"✅ Opération bulk terminée")
if response.get("errors"):
    print("❌ Certains documents ont échoué:")
    for item in response["items"]:
        if "error" in item.get("index", {}):
            print(f"  - ID {item['index'].get('_id')}: {item['index']['error']}")
else:
    print(f"✅ {len(response['items'])} documents insérés avec succès")


## 5. Suppression d'un document

Supprimons un document spécifique par son ID.


In [ ]:
# Supprimer le document avec l'ID "1"
response = es.delete(index="tweet", id="1")
print("✅ Document supprimé")
pprint(response)


## 6. Recherche dans l'index

Effectuons une recherche simple dans l'index pour récupérer tous les documents.


In [ ]:
# Recherche simple (récupère tous les documents)
response = es.search(index="tweet", body={})
print(f"✅ {response['hits']['total']['value']} document(s) trouvé(s)")
print("\nRésultats:")
for hit in response['hits']['hits']:
    print(f"\nID: {hit['_id']}")
    pprint(hit['_source'])


## 6. Agrégation : Compter les tweets par président

Les agrégations permettent d'analyser les données. Ici, nous utilisons une agrégation `terms` pour compter le nombre de tweets par `user_name`.


In [ ]:
# Recherche avec agrégation pour compter les tweets par président
response = es.search(
    index="tweet",
    body={
        "aggs": {
            "presidents": {
                "terms": {
                    "field": "user_name"
                }
            }
        }
    }
)

print("✅ Résultats de l'agrégation:")
print("\nNombre de tweets par président:")
for bucket in response['aggregations']['presidents']['buckets']:
    print(f"  - {bucket['key']}: {bucket['doc_count']} tweet(s)")


## 7. Création d'un nouvel index : private_message

Créons un second index pour stocker des messages privés.


In [ ]:
# Supprimer l'index s'il existe déjà
if es.indices.exists(index="private_message"):
    es.indices.delete(index="private_message")
    print("🗑️  Ancien index 'private_message' supprimé")

# Créer le nouvel index
es.indices.create(
    index="private_message",
    body={
        "settings": {
            "index": {
                "number_of_shards": 1,
                "number_of_replicas": 1
            }
        }
    }
)
print("✅ Index 'private_message' créé avec succès")


## 7. Ajout d'un document dans private_message

Ajoutons un document dans le nouvel index.


In [ ]:
# Ajouter un document dans l'index private_message
response = es.index(
    index="private_message",
    document={
        "user_name_a": "Poutine",
        "user_name_b": "Trump",
        "content": "I miss you Donald",
        "tweeted_at": "2022-03-16"
    }
)
print("✅ Document ajouté dans 'private_message'")
pprint(response)


## 8. Création d'un alias

Un alias permet de regrouper plusieurs index sous un même nom. C'est très utile pour interroger plusieurs index simultanément.


In [ ]:
# Créer un alias "flux" qui pointe vers les index "private_message" et "tweet"
es.indices.put_alias(
    index=["private_message", "tweet"],
    name="flux"
)
print("✅ Alias 'flux' créé pour les index 'private_message' et 'tweet'")


## 9. Test de l'alias

Testons l'alias en effectuant une recherche sur "flux", qui devrait retourner les documents des deux index.


In [ ]:
# Recherche via l'alias "flux"
response = es.search(index="flux", body={})
print(f"✅ {response['hits']['total']['value']} document(s) trouvé(s) via l'alias 'flux'")
print("\nRésultats:")
for hit in response['hits']['hits']:
    print(f"\nIndex: {hit['_index']}, ID: {hit['_id']}")
    pprint(hit['_source'])


## Nettoyage

Supprimons l'index private_message pour nettoyer (optionnel).


In [ ]:
# Supprimer l'index private_message
if es.indices.exists(index="private_message"):
    es.indices.delete(index="private_message")
    print("🗑️  Index 'private_message' supprimé")
else:
    print("ℹ️  L'index 'private_message' n'existe pas")


## Résumé des opérations

Ce notebook a couvert les principales opérations Elasticsearch :

1. ✅ Vérification de la santé du cluster
2. ✅ Fix des paramètres d'index en lecture seule
3. ✅ Création d'index avec paramètres
4. ✅ Définition de mappings
5. ✅ Ajout de documents (simple et bulk)
6. ✅ Suppression de documents
7. ✅ Recherche dans les index
8. ✅ Agrégations (analyse de données)
9. ✅ Création et utilisation d'alias

Toutes ces opérations peuvent être exécutées via l'API REST d'Elasticsearch ou via la librairie Python `elasticsearch`.
